<a href="https://colab.research.google.com/github/Ricollis/w209_final_project/blob/main/Matt_Final_Project_w209_JC_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Load libraries
import http.client
import requests
import json
import pandas as pd
import csv
import time
from google.colab import files
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#Download zip code csv to your hard drive from here
#https://www.downloadexcelfiles.com/us_en/download-list-zip-codes-florida-state-united-states#.YqpRQezML0o

#Upload zipcodes 
files.upload()

zipcodes = pd.read_csv("usa_zipcode_of_fl-1526j.csv")

zipcodes.head()

Saving usa_zipcode_of_fl-1526j.csv to usa_zipcode_of_fl-1526j.csv


,zip,Zipcode name,City,State,County Name
0,32615,"ALACHUA, FL",ALACHUA,FL,ALACHUA
1,32616,"ALACHUA, FL",ALACHUA,FL,ALACHUA
2,32420,"ALFORD, FL",ALFORD,FL,JACKSON
3,32701,"ALTAMONTE SPRINGS, FL",ALTAMONTE SPRINGS,FL,SEMINOLE
4,32714,"ALTAMONTE SPRINGS, FL",ALTAMONTE SPRINGS,FL,SEMINOLE


In [ ]:
zipcodes[zipcodes['City'] == 'JACKSONVILLE']

,zip,Zipcode name,City,State,County Name
493,32099,"JACKSONVILLE, FL",JACKSONVILLE,FL,DUVAL
494,32201,"JACKSONVILLE, FL",JACKSONVILLE,FL,DUVAL
495,32202,"JACKSONVILLE, FL",JACKSONVILLE,FL,DUVAL
496,32203,"JACKSONVILLE, FL",JACKSONVILLE,FL,DUVAL
497,32204,"JACKSONVILLE, FL",JACKSONVILLE,FL,DUVAL
498,32205,"JACKSONVILLE, FL",JACKSONVILLE,FL,DUVAL
499,32206,"JACKSONVILLE, FL",JACKSONVILLE,FL,DUVAL
500,32207,"JACKSONVILLE, FL",JACKSONVILLE,FL,DUVAL
501,32208,"JACKSONVILLE, FL",JACKSONVILLE,FL,DUVAL
502,32209,"JACKSONVILLE, FL",JACKSONVILLE,FL,DUVAL


In [ ]:
#Set API url
url = "https://zillow-com1.p.rapidapi.com/propertyExtendedSearch"

#Set API key and host information
headers = {
	"X-RapidAPI-Key": "c3125abd81msh24dd1570f7ae084p10efe9jsn85b6774ecedd",
	"X-RapidAPI-Host": "zillow-com1.p.rapidapi.com"
}

# Clear out df so we don't get duplicates
df = None

# Set number of desired zip codes to request
num_zip_codes = 50

# Loop through zip codes to grab homes 
for index, row in zipcodes[:num_zip_codes].iterrows():

    # Construct and execute API query for Zip code
    page = 1
    print('ZIP:', row['zip'], 'Name:', row['Zipcode name'], 'Page',page)
    querystring = {"location":row['zip'],"home_type":"Houses", "page": page}
    response = json.loads(requests.request("GET", url, headers=headers, params=querystring).text)
    time.sleep(.33)
    # Determine how many pages of results there are for the current ZIP
    if 'totalPages' in response:
      pages_per_zip = response['totalPages']
    else:
      # If the response doesn't have the right keys, skip this ZIP
      continue

    # Get the results for as many pages there are for the current ZIP
    for page in range(2, pages_per_zip + 1):
      print('ZIP:', row['zip'], 'Name:', row['Zipcode name'], 'Page',page)
      querystring = {"location":row['zip'],"home_type":"Houses", "page": page}
      response = json.loads(requests.request("GET", url, headers=headers, params=querystring).text)
      time.sleep(.33)
      # Only add to the df if the ZIP response has results
      if response['totalResultCount'] > 0:
        df = pd.concat([df, pd.DataFrame(response['props'])])

    # Create the df if it doesn't exist, else add to it
    if df is None:
      if response['totalResultCount'] > 0:
        df = pd.DataFrame(response['props'])
    else:
      if response['totalResultCount'] > 0:
        df = pd.concat([df, pd.DataFrame(response['props'])])

ZIP: 32236 Name: JACKSONVILLE, FL Page 3
ZIP: 32236 Name: JACKSONVILLE, FL Page 4
ZIP: 32237 Name: JACKSONVILLE, FL Page 1
ZIP: 32238 Name: JACKSONVILLE, FL Page 1


KeyboardInterrupt: ignored

In [ ]:
url = "https://zillow-com1.p.rapidapi.com/propertyExtendedSearch"

#Set API key and host information
headers = {
	"X-RapidAPI-Key": "c3125abd81msh24dd1570f7ae084p10efe9jsn85b6774ecedd",
	"X-RapidAPI-Host": "zillow-com1.p.rapidapi.com"
}
page = 1
querystring = {"location":"32237","home_type":"Houses", "page": page}
response = json.loads(requests.request("GET", url, headers=headers, params=querystring).text)

In [ ]:
response

[{'zpid': 45367112},
 {'zpid': 334528305},
 {'zpid': 299181441},
 {'zpid': 50761395},
 {'zpid': 24995446}]

In [ ]:
#Place data into CSV files
df.to_csv('/content/gdrive/My Drive/fl_houses.csv')

In [ ]:
df.columns

NameError: ignored